In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import uproot
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import os
import optparse
import os.path as osp
import awkward as ak
import math
import torch_geometric
import torch
import sys
#import pandas as pd
from torch_geometric.data import Data
import glob

/home/llr/cms/sghosh/software/miniconda3/envs/torchtest3p8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/llr/cms/sghosh/software/miniconda3/envs/torchtest3p8/lib/python3.8/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: libc10_cuda.so: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/llr/cms/sghosh/software/miniconda3/envs/torchtest3p8/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: libc10_cuda.so: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-spa

In [3]:
def analyze(filename,nfile,threshold, output_dir):



        filein = uproot.open(filename)["ticlDumper"]
        
        
        
        clusters = filein["clusters"].arrays()
        clusters = clusters[[x for x in ak.fields(clusters) if x not in ["run_","luminosityBlock_","event_"]]]
        tracksters = filein["trackstersCLUE3DHigh"].arrays()
        associations = filein["associations"].arrays()
        simtrackstersCP = filein["simtrackstersCP"].arrays()

        ## flatten to get better shapes
        simToReco_CP = ak.flatten(associations.tsCLUE3D_simToReco_CP)
        simToReco_CP_score = ak.flatten(associations.tsCLUE3D_simToReco_CP_score)

        ## choose the one with best score | best for closeby photons/electrons 
        minsimscore_idxs = ak.argmin(simToReco_CP_score,axis=-1,keepdims=True )


        ## get corrsp trkster idxs
        matchtrk_idxs = simToReco_CP[minsimscore_idxs]

        tracksters.raw_energy[matchtrk_idxs]

        ratio_trksim = tracksters.raw_em_energy[matchtrk_idxs]/simtrackstersCP.raw_em_energy
        ratio_trksimtot = tracksters.raw_energy[matchtrk_idxs]/simtrackstersCP.raw_energy
        #ratio_trksim = ratio_trksim.to_numpy()

        #sel_clusters = clusters[tracksters.vertices_indexes[matchtrk_idxs]]
        ncalop = len(simtrackstersCP)



        listoftensors = []
        ##maskcalo = ak.argmin(sel_simmatch_scr[sel_simmatch_idxs == 0],axis=1,keepdims=True)
        for calop in range(ncalop):
            ntrks = len(tracksters.raw_energy[matchtrk_idxs][calop])
            
            if ak.is_none(matchtrk_idxs[calop]):
                #print("was isnone skipped")
                continue
                
            #print(ntrks)
            for ntrk in range(ntrks):
                gun_feats = np.stack((simtrackstersCP.boundaryEta[calop],
                                      simtrackstersCP.boundaryPhi[calop],
                                      simtrackstersCP.raw_em_energy[calop],
                                      simtrackstersCP.raw_energy[calop], 
                                      ratio_trksim[calop],
                                      ratio_trksimtot[calop])).to_numpy().data.T

                #print(gun_feats.shape)
                #print(gun_feats[:,3])

                clus2d_feats = np.stack((ak.to_numpy(clusters.position_x[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.position_y[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.position_z[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.energy[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.cluster_time[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]),
                                 ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)]))).T


                #print(clus2d_feats.shape)
                #print(np.sum(clus2d_feats[:,3]))


                clus3d_feats = np.stack((tracksters.barycenter_eta[matchtrk_idxs][calop][ntrk],
                                         tracksters.barycenter_phi[matchtrk_idxs][calop][ntrk],
                                         tracksters.raw_em_energy[matchtrk_idxs][calop][ntrk],
                                         tracksters.raw_energy[matchtrk_idxs][calop][ntrk],
                                         min(ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)])),
                                         max(ak.to_numpy(clusters.cluster_layer_id[calop][ak.flatten(tracksters.vertices_indexes[matchtrk_idxs][calop][ntrk], axis=-1)])))).T


                #print(clus3d_feats.shape)
                #print(clus3d_feats[3])


                datum = Data(clus2d_feat = torch.tensor(clus2d_feats, dtype=torch.float32),
                            clus3d_feat = torch.tensor(clus3d_feats, dtype=torch.float32),
                            gun_feat = torch.tensor(gun_feats, dtype=torch.float32))

                listoftensors.append(datum)
                    
                    

        processed_dir = output_dir #'/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_DATA/PHO/' ## PHO
        import os
        import os.path as osp
        if not os.path.exists(processed_dir):
            os.makedirs(processed_dir)


        #torch.save(listoftensors, osp.join(processed_dir, 'data_{}_{}_{}_{}.pt'.format(nfile,evt,ngun,trkidx)))
        torch.save(listoftensors, osp.join(processed_dir, 'data_{}.pt'.format(nfile)))
        #print("skipped events:",nsskip)
        #print("average #tracksters:",np.mean(ntrkster))
        return 0

In [4]:
### LOW ENERGY PHOTONS

#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240827_photonsD98_2-15GeV_step3/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_TICLDUMPER_DATA/S2Rmin_pho2to15_27092024/'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")

['step3_dumper_1.root', 'step3_dumper_3.root', 'step3_dumper_6.root', 'step3_dumper_5.root', 'step3_dumper_4.root', 'step3_dumper_2.root', 'step3_dumper_8.root', 'step3_dumper_9.root', 'step3_dumper_7.root', 'step3_dumper_10.root', 'step3_dumper_11.root', 'step3_dumper_12.root', 'step3_dumper_13.root', 'step3_dumper_15.root', 'step3_dumper_16.root', 'step3_dumper_18.root', 'step3_dumper_19.root', 'step3_dumper_20.root', 'step3_dumper_14.root', 'step3_dumper_21.root', 'step3_dumper_17.root', 'step3_dumper_22.root', 'step3_dumper_23.root', 'step3_dumper_26.root', 'step3_dumper_25.root', 'step3_dumper_24.root', 'step3_dumper_28.root', 'step3_dumper_33.root', 'step3_dumper_32.root', 'step3_dumper_29.root', 'step3_dumper_34.root', 'step3_dumper_35.root', 'step3_dumper_36.root', 'step3_dumper_30.root', 'step3_dumper_27.root', 'step3_dumper_31.root', 'step3_dumper_38.root', 'step3_dumper_37.root', 'step3_dumper_41.root', 'step3_dumper_42.root', 'step3_dumper_39.root', 'step3_dumper_43.root', 

300it [1:01:29, 12.30s/it]


In [4]:
#raw_dir='/grid_mnt/data__data.polcms/cms/tarabini/GENPHOTESTPU2_noSmearing/step3_ticlv4_clue3d_CMSSW_12_4_0_pre2/'
#raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240419_pions_step3/" ### PI
raw_dir="/grid_mnt/data__data.polcms/cms/cappati/240517_photonsD98_step3/"
output_dir = '/grid_mnt/data__data.polcms/cms/sghosh/NEWPID_TICLDUMPER_DATA/S2Rmin_pho_30072024/'
fnamelist = os.listdir(raw_dir)#+'STEP3_*.root'
print(fnamelist)

for idx, file in tqdm(enumerate(fnamelist)):
    try:
        #print(raw_dir+file,idx)
        analyze(raw_dir+file,idx, 0.6, output_dir) ### PHO
        #analyze(raw_dir+file,idx, 0.99) ### PI
        #break
    except:
        print("file is invalid, skipping....")

['step3_dumper_8.root', 'step3_dumper_1.root', 'step3_dumper_5.root', 'step3_dumper_14.root', 'step3_dumper_10.root', 'step3_dumper_2.root', 'step3_dumper_4.root', 'step3_dumper_18.root', 'step3_dumper_6.root', 'step3_dumper_7.root', 'step3_dumper_3.root', 'step3_dumper_17.root', 'step3_dumper_11.root', 'step3_dumper_15.root', 'step3_dumper_24.root', 'step3_dumper_12.root', 'step3_dumper_9.root', 'step3_dumper_28.root', 'step3_dumper_20.root', 'step3_dumper_25.root', 'step3_dumper_16.root', 'step3_dumper_34.root', 'step3_dumper_13.root', 'step3_dumper_29.root', 'step3_dumper_26.root', 'step3_dumper_38.root', 'step3_dumper_19.root', 'step3_dumper_27.root', 'step3_dumper_43.root', 'step3_dumper_37.root', 'step3_dumper_48.root', 'step3_dumper_21.root', 'step3_dumper_23.root', 'step3_dumper_30.root', 'step3_dumper_31.root', 'step3_dumper_45.root', 'step3_dumper_42.root', 'step3_dumper_47.root', 'step3_dumper_41.root', 'step3_dumper_32.root', 'step3_dumper_33.root', 'step3_dumper_36.root', 

500it [1:45:20, 12.64s/it]
